In [ ]:
# Created by Anil Burak Karadede (https://www.linkedin.com/in/anilburakkaradede/)
pip install google-ads

# Google Ads API

In [5]:
from google.ads.googleads.client import GoogleAdsClient

In [6]:
googleads_client = GoogleAdsClient.load_from_storage("google-ads.yaml")

In [7]:
import sys
from google.ads.googleads.client import GoogleAdsClient
from google.ads.googleads.errors import GoogleAdsException

_DEFAULT_LOCATION_IDS = ["2792"] # türkiye: ["2792"], "1037" | | İngiltere: ["2826"], "1000"  Other country codes: https://developers.google.com/google-ads/api/data/geotargets | | Other Language Codes: https://developers.google.com/google-ads/api/data/codes-formats
_DEFAULT_LANGUAGE_ID = "1037"

def main(
    client, customer_id, location_ids, language_id, keyword_texts, page_url
):
    keyword_plan_idea_service = client.get_service("KeywordPlanIdeaService")
    keyword_competition_level_enum = (
        client.enums.KeywordPlanCompetitionLevelEnum
    )
    
    keyword_annotation = [(
        client.enums.KeywordPlanKeywordAnnotationEnum.KEYWORD_CONCEPT
    )]
    
    keyword_plan_network = (
        client.enums.KeywordPlanNetworkEnum.GOOGLE_SEARCH
    )
    
    location_rns = map_locations_ids_to_resource_names(client, location_ids)
    language_rn = client.get_service("GoogleAdsService").language_constant_path(
        language_id
    )

    if not (keyword_texts or page_url):
        raise ValueError(
            "At least one of keywords or page URL is required, "
            "but neither was specified."
        )
        
    request = client.get_type("GenerateKeywordIdeasRequest")
    request.customer_id = customer_id
    request.language = language_rn
    request.geo_target_constants = location_rns
    request.include_adult_keywords = False
    request.keyword_plan_network = keyword_plan_network
    request.keyword_annotation = keyword_annotation

    if not keyword_texts and page_url:
        request.url_seed.url = page_url

    if keyword_texts and not page_url:
        request.keyword_seed.keywords.extend(keyword_texts)

    if keyword_texts and page_url:
        request.keyword_and_url_seed.url = page_url
        request.keyword_and_url_seed.keywords.extend(keyword_texts)

    keyword_ideas = keyword_plan_idea_service.generate_keyword_ideas(
        request=request
    )

    keyword_ideas = list(keyword_ideas)
    formatted_data = {}
    exact_keyword = [{
            "monthlysearch": keyword_ideas[0].keyword_idea_metrics.avg_monthly_searches,
            "keyword": keyword_ideas[0].text,
            "difficulty": keyword_ideas[0].keyword_idea_metrics.competition.name,
            "competition_score":keyword_ideas[0].keyword_idea_metrics.competition_index,
            "annotation" : str(keyword_ideas[0].keyword_annotations)
        }]
    formatted_data['exact_keyword'] = (exact_keyword)
    related_keywords = []
    for idea in keyword_ideas[1:]:
        competition_value = idea.keyword_idea_metrics.competition.name
        related_keywords.append({
            "keyword": idea.text,
            "monthlysearch": idea.keyword_idea_metrics.avg_monthly_searches,
            "difficulty": competition_value,
            "competition_score":idea.keyword_idea_metrics.competition_index,
            "annotation" : str(idea.keyword_annotations)
        })
    formatted_data['related_keywords'] = related_keywords
    return formatted_data

def map_locations_ids_to_resource_names(client, location_ids):
    build_resource_name = client.get_service(
        "GeoTargetConstantService"
    ).geo_target_constant_path
    return [build_resource_name(location_id) for location_id in location_ids]

def start(keyword):

    googleads_client = GoogleAdsClient.load_from_storage("google-ads.yaml")

    customer_id = 'xxxxxxxxxx' #Enter the account number of your Google Ads test account without -.
    keyword_texts = [keyword]
    location_ids = _DEFAULT_LOCATION_IDS
    language_id = _DEFAULT_LANGUAGE_ID
    page_url = None

    try:
        return main(
            googleads_client,
            customer_id, 
            location_ids,
            language_id,
            keyword_texts,
            page_url,
        )

    except GoogleAdsException as ex:
        print(
            f'Request with ID "{ex.request_id}" failed with status '
            f'"{ex.error.code().name}" and includes the following errors:'
        )
        for error in ex.failure.errors:
            print(f'\tError with message "{error.message}".')
            if error.location:
                for field_path_element in error.location.field_path_elements:
                    print(f"\t\tOn field: {field_path_element.field_name}")
        sys.exit(1)

In [8]:
import json
import re

def convert_to_dict(input_str):
    pattern = r'concepts {\n  name: "(.*?)"\n  (concept_group {\n    .*?\n  }\n)}'
    matches = re.findall(pattern, input_str, re.DOTALL)
    
    result = {"concepts": []}
    for match in matches:
        name = match[0]
        concept_group_str = match[1]

        
        pattern = r'name: "(.*?)"\n(    type_: (.*?)\n)?'
        concept_group_matches = re.findall(pattern, concept_group_str, re.DOTALL)
        for cg_match in concept_group_matches:
            concept_group = {"name": cg_match[0]}
            if cg_match[1].strip():
                concept_group["type_"] = cg_match[1].strip()

        concept = {"name": name, "concept_group": concept_group}
        result["concepts"].append(concept)
    
    return result["concepts"]




def keyword_row_parser(data):
    """parse each response into a set of rows.""";
    
    final_row = []
    base_keyword = data["exact_keyword"][0]["keyword"]
    base_montlysearch = data["exact_keyword"][0]["monthlysearch"]
    based_difficulty = data["exact_keyword"][0]["difficulty"]
    base_competition_score = data["exact_keyword"][0]["competition_score"]
    base_annotation = data["exact_keyword"][0]["annotation"]
    
    initial_data = {
                    "base_keyword": base_keyword,
                    "base_montly_search": base_montlysearch,
                    "based_difficulty": based_difficulty,
                    "base_competition_score": base_competition_score,
                    "related_keyword": base_keyword,
                    "rk_monthly_search":base_montlysearch,
                    "difficulty": based_difficulty,
                    "competition_score": base_competition_score,
                    "annotation": base_annotation,
                    "name":None,
                    "concept_name": None,
                    "concept_type": None,
                    }
            
    final_row.append(initial_data)
    
    for item in data["related_keywords"]:
        related_keyword = item["keyword"]
        rk_monthly_search = item["monthlysearch"]
        difficulty = item["difficulty"]
        competition_score = item["competition_score"]
        
        annotation = item["annotation"]
        
        try:
            concept_list = convert_to_dict(annotation)
            for item in concept_list:
                
                name = item["name"]
                concept_name = item["concept_group"]["name"]
                concept_type = item["concept_group"]["type_"]

                data = {
                    "base_keyword": base_keyword,
                    "base_montly_search": base_montlysearch,
                    "based_difficulty": based_difficulty,
                    "base_competition_score": base_competition_score,
                    "related_keyword": related_keyword,
                    "rk_monthly_search":rk_monthly_search,
                    "difficulty": difficulty,
                    "competition_score": competition_score,
                    "annotation": annotation,
                    "name":name,
                    "concept_name": concept_name,
                    "concept_type": concept_type,
                    }
                
                final_row.append(data)
            
        except:
            continue
            
            
    return final_row

In [29]:
import pandas as pd

#Enter the name of your CSV file 👇 
input_data = pd.read_csv("dosya_adi.csv", header=None) 

seed_terms = [x for x in input_data[0]]

In [ ]:
from google.ads.googleads.client import GoogleAdsClient
import time

client = GoogleAdsClient.load_from_storage("google-ads.yaml")
list_keywords = []
failed_terms = []  
retry_count = 5


for x in seed_terms[0:1000]:
    retries = 0
    while retries < retry_count:
        try:
            print(f"Seed terim {x} için {retries + 1}. deneme API isteği gönderiliyor...") 
            result = main(client, "xxxxxxxxxx", ["2792"], "1037", [x], None) #Enter the account number of your Google Ads test account without -. Language and country code for Turkey: ["2792"], "1037"
            print(f"Seed terim {x} için API isteği başarılı.")  
            parse_result = keyword_row_parser(result)
            list_keywords.extend(parse_result)
            break 
        except Exception as e:
            print(f"Hata: {e}, Terim: {x}, Deneme: {retries + 1}")
            retries += 1
            time.sleep(3)

    if retries == retry_count:
        failed_terms.append(x)

print("Hata alan seed terimleri:", failed_terms)


for x in failed_terms[0:1000]:
    retries = 0
    while retries < retry_count:
        try:
            print(f"Hatayla karşılaşan terim {x} için {retries + 1}. deneme API isteği gönderiliyor...") 
            result = main(client, "xxxxxxxxxx", ["2792"], "1037", [x], None) #Enter the account number of your Google Ads test account without -. Language and country code for Turkey: ["2792"], "1037"
            print(f"Hatayla karşılaşan terim {x} için API isteği başarılı.") 
            parse_result = keyword_row_parser(result)
            list_keywords.extend(parse_result)
            break  
        except Exception as e:
            print(f"Hata: {e}, Terim: {x}, Deneme: {retries + 1}")
            retries += 1
            time.sleep(4)  

    if retries == retry_count:
        print(f"Terim {x} için tekrar deneme başarısız.")

print("Tüm seed terimleri için API verileri:", list_keywords)

In [32]:
output_data = pd.DataFrame(list_keywords)

# Google Suggestions

It takes Google's autocomplete suggestions by adding the letters and numbers "abcdefghijklmnopqrstuvwxyz0123456789" one by one at the beginning and end of each keyword you enter for keyword research.

In [1]:
import concurrent.futures
import pandas as pd
import itertools
import requests
import string
import json
import time

In [2]:
startTime = time.time()

In [70]:
WAIT_TIME = 0.1
MAX_WORKERS = 20

# Enter the language you want to target below 👇
lang = "tr"

charList = " " + string.ascii_lowercase + string.digits

def makeGoogleRequest(query):
     
    time.sleep(WAIT_TIME)
    URL="https://google.com/complete/search"   
    PARAMS = {"client":"firefox",
            "hl":lang,
            "q":query}
    headers = {'User-agent':'Mozilla/5.0'}
    response = requests.get(URL, params=PARAMS, headers=headers)
    if response.status_code == 200:
        suggestedSearches = json.loads(response.content.decode('utf-8'))[1]
        return suggestedSearches
    else:
        return "ERR"

def getGoogleSuggests(keyword):
    
    queryList = [keyword + " " + char for char in charList] + [char + " " + keyword for char in charList]
    suggestions = []
    for query in queryList:
        suggestion = makeGoogleRequest(query)
        if suggestion != 'ERR':
            suggestions.append(suggestion)

    
    suggestions = set(itertools.chain(*suggestions))
    if "" in suggestions:
        suggestions.remove("")

    return suggestions



keywords = seed_terms 

resultList = []

with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futuresGoogle = {executor.submit(getGoogleSuggests, keyword): keyword for keyword in keywords}

    for future in concurrent.futures.as_completed(futuresGoogle):
        key = futuresGoogle[future]
        for suggestion in future.result():
            resultList.append([key, suggestion])


outputDf = pd.DataFrame(resultList, columns=['Keyword','Suggestion'])

In [62]:


final = pd.DataFrame()


final['main_kw'] = pd.concat([output_data['base_keyword'], outputDf['Keyword']], ignore_index=True)


final['related_keyword'] = pd.concat([output_data['related_keyword'], outputDf['Suggestion']], ignore_index=True)

# Gemini Semantic Keywords

https://aistudio.google.com/app/prompts/new_chat upload the sheet to the address as in the example format below and run the following prompt. The prompt asks for 5 keyword suggestions for each word. You can increase this number if you want.

Sample CSV format to be transmitted to Gemini: https://docs.google.com/spreadsheets/d/19yzrez75KQmZ7W0iFB6MPooSxEDjiuzjEdhPfXxScXc/edit?usp=sharing 

Prompt 👇

*This CSV file has two columns, Keywords and Related Keywords. For each word in the Keywords column, I want you to write five semantic words that mean the same thing as these words in the Related Keywords column opposite it. These words should be both related and related to how else people might search for these words in Google. You should write these words side by side in the relevant row, separated by commas. Don't write any extra explanations. Just write the words and send me the edited csv file again.*

Gemini will return you a response as in the picture at https://prnt.sc/vdqD80gd6JnI. Here, starting from the Keywords section, load the relevant part into the data = field in the code block below. You can view the sample format below. You can delete this part and copy and paste the response from Gemini in the same format into this field.

In [64]:


import pandas as pd

data = """Keywords,Related Keywords
arkadaşlarla oynanacak oyunlar,çok oyunculu oyunlar, çevrimiçi oyunlar, arkadaşlarla oynanabilecek oyunlar, birlikte oynanacak oyunlar, multiplayer oyunlar
klavye kısayolları, klavyeden kısayollar, klavye hileleri, kısayol tuşları, pc kısayolları, bilgisayar kısayolları
google hileleri, google arama hileleri, google aramaları, google arama motorunda hileler, google aramaları için hileler, google search hileleri
instagram keşfet sıfırlama, instagram keşfet algoritması, instagram keşfet algoritması sıfırlama, instagram keşfet sıfırla, instagram keşfet feed'ini sıfırla, instagram keşfet algoritmasını sıfırla
instagram kullanıcı adı değiştirme, instagramda kullanıcı adı değiştirme, instagram username değiştirme, instagram kullanıcı adı nasıl değiştirilir, instagramda kullanıcı adı nasıl değiştirilir, instagramda kullanıcı adı değiştirme işlemi
wifi şifresi öğrenme, wifi şifresini öğrenme, wifi şifresi bulma, wifi şifresini görme, kablosuz şifre bulma, wifi ağ şifresi bulma
vsync nedir, vsync açıklaması, vsync ne işe yarar, vsync ayarları, vsync özelliği, vsync ne yapar
oem nedir, oem lisans, oem ürün, oem yazılım, oem marka, oem versiyon
ip adresi değiştirme, ip değiştirme, ip adresi gizleme, ip adresi değiştirme programı, ip adresi nasıl değiştirilir, ip adresi değiştirme yöntemi
whatsapp yedeklenen mesajları geri getirme, whatsapp yedeğini geri yükleme, whatsapp yedek mesajları geri yükleme, whatsapp mesajları geri yükleme, whatsapp silinen mesajları geri getirme, whatsapp yedekten geri yükleme
mac adresi nedir, mac adresi ne işe yarar, mac adres nasıl bulunur, mac adresi öğrenme, mac adresi değiştirme, mac adresi ne demek
tarayıcı oyunları, internet oyunları, çevrimiçi oyunlar, online oyunlar, browser oyunları, web oyunları
telefon ekranını bilgisayara yansıtma, telefon ekranı yansıtma, telefon ekranı paylaşma, telefon ekranı bilgisayara gösterme, telefon ekranı aktarma, telefon ekranı bilgisayar ile senkronize etme
açık dünya oyunları, sandbox oyunlar, serbest oyunlar, açık alan oyunları, üç boyutlu oyunlar, rol yapma oyunları
steam hesap değeri öğrenme, steam hesap değeri nasıl hesaplanır, steam hesap değeri kontrol etme, steam hesap değeri kontrolü, steam hesap değeri hesaplama, steam hesabı değerlendirme
silinen dosyaları geri getirme nasıl yapılır, silinen dosyaları geri yükleme, silinen dosyaları kurtarma, silinen dosyaları geri getirme programı, silinen dosyaları geri getirme yöntemleri, silinen dosyaları geri kazandırma
internetsiz oyunlar, offline oyunlar, tek oyunculu oyunlar, çevrimdışı oyunlar, bilgisayar oyunları, mobil oyunlar
whatsapp plus, whatsapp mod, whatsapp mod apk, whatsapp plus apk, whatsapp plus indir, whatsapp mod indir
tbt ne demek, tbt anlamı, tbt ne anlama geliyor, tbt ne anlama gelir, tbt kısaltması, tbt nedir
whatsapp silinen mesajları geri getirme, whatsapp yedeğinden mesaj geri yükleme, whatsapp silinen mesajları kurtarma, whatsapp silinen mesajları geri getirme programı, whatsapp silinen mesajları geri getirme yöntemi, whatsapp silinen mesajları geri yükleme
"""


rows = data.strip().split("\n")


keywords = []
related_keywords = []
for row in rows[1:]:
    keyword, related = row.split(",", 1)
    keywords.append(keyword)
    related_keywords.append(related)


df = pd.DataFrame({"Keywords": keywords, "Related Keywords": related_keywords})


df.index = range(len(df))


new_df = pd.DataFrame(columns=["Keywords", "Related Keywords"])
current_keyword = ""
for i, row in df.iterrows():
    if i == 0:
        current_keyword = row["Keywords"]
    for related_keyword in row["Related Keywords"].split(","):
        new_df = pd.concat([new_df, pd.DataFrame({"Keywords": [current_keyword], "Related Keywords": [related_keyword.strip()]})], ignore_index=True)
    if i < len(df) - 1:
        current_keyword = df.loc[i+1, "Keywords"]

In [66]:


new_data = pd.DataFrame({
    'main_kw': new_df['Keywords'],
    'related_keyword': new_df['Related Keywords']
})

final = pd.concat([final, new_data], ignore_index=True)


# Delete Duplicate Rows in Dataframe and Export All Data in XLSX Format

In [ ]:
original_row_count = final.shape[0]

final = final.drop_duplicates()

rows_deleted = original_row_count - final.shape[0]

print(f"Silinen satır sayısı: {rows_deleted}")

In [58]:
final.to_excel('dosya_adi.xlsx') # 👈 Specify the file name you want to export all this data to

Copy and paste all keywords in the related_keyword column of this xlsx file into the input field at https://contentgecko.io/free-serp-keyword-clustering. Then select the target country, language and minimum SERP similarity and press the create clusters button. When you enter your e-mail address on the next screen, clustered data will be sent to your e-mail address within two minutes on average. https://prnt.sc/6un7vv3juTlt * 5000 keywords can be clustered at a time. There are no restrictions other than that.

Different clustering algorithms can be used here. However, since this tool does the clustering process according to the SERP, we actually act completely according to what Google understands from the relevant search terms. 

# Parse a Clustered Keyword File

Cluster results will be sent to your e-mail address in xlsx format. (https://prnt.sc/5AfOEJByPHbt) Download this file to your computer and enter the file path in the code block below. This code block will process the xlsx file and extract all keywords belonging to the cluster group in which the keyword you have specified is in. In this way, keywords that are not related to the keyword will be eliminated and you will be able to access the final data. 

In [101]:
import pandas as pd

# Upload the CSV file sent to your e-mail address from contentgecko. 👇
df = pd.read_excel('contentgecko-10060-2024-06-17.xlsx') 


aranan_kelimeler = seed_terms
  


changed_rows = set()


for kelime in aranan_kelimeler:
    
    matching_rows = df[df['keyword'] == kelime]
    
    
    clusters_to_change = matching_rows['cluster'].unique()
    
    
    for cluster_value in clusters_to_change:
        
        indices = df.loc[df['cluster'] == cluster_value].index
        df.loc[indices, 'cluster'] = kelime
        
        
        changed_rows.update(indices)


df_changed = df.loc[sorted(changed_rows)]
df_changed = df_changed[df_changed['search_volume'] >= 10] # It deletes those with a search volume of less than 10. You can increase this value if you want.

# Enter a file name to save the results in a new CSV file 👇
df_changed.to_csv('dosya_adi.csv', index=False)

# Created by Anil Burak Karadede (https://www.linkedin.com/in/anilburakkaradede/)